<h2>Taller 4 - Pick and place usando cinemática inversa</h2>
<br>Autor: Claudio Morales D.
<br>Centro de Automatización y Robótica Inacap
<br>Santiago, Chile, enero 2023


Desde CoppeliaSim, abrir la escena 'scara_pick_n_place.ttt'
<br>Los conceptos de cinemática directa y inversa pueden consultarse en este enlace: https://youtu.be/G2A2Zm7BfXY
<br>En una actividad previa se muestra la obtención de la cinemática directa del robot: https://youtu.be/ZiRuWQo7EzA 

<h3>1. Nos conectamos y enviamos al robot a una posición inicial deseada</h3>

In [ ]:
# Cargamos la librería, creamos un cliente y obtenemos acceso a sim
import zmqRemoteApi
client = zmqRemoteApi.RemoteAPIClient()
sim = client.getObject('sim')

# Obtenemos los manejadores para las articulaciones y el actuador final
j1 = sim.getObject('/base/joint1')
j2 = sim.getObject('/base/joint2')
j3 = sim.getObject('/base/joint3')
suctionPad = sim.getObject('/base/suctionPad')

print(j1, j2, j3, suctionPad)

# Enviamos al robot la posición deseada
q1_val = 0 * 3.1416/180
q2_val = 0.2
q3_val = 0 * 3.1416/180

sim.setJointTargetPosition(j1, q1_val)
sim.setJointTargetPosition(j2, q2_val)
sim.setJointTargetPosition(j3, q3_val)

<h3>2. Verificamos que el suctionPad funciona bien</h3>

In [ ]:
def setEffector(val):
# acciona el efector final (suctionPad)
# val es Int con valor 0 ó 1 para desactivar o activar el actuador final.
    objectHandle = sim.getObject('/base/suctionPad')
    scriptHandle = sim.getScript(sim.scripttype_childscript,objectHandle)
    res = sim.callScriptFunction('setEffector',scriptHandle,val)
    return res

In [ ]:
#verificamos si el Pad se activa y desactiva
setEffector(0)

In [ ]:
# 1. Obtenemos el manejador de la caja 1 y la situamos bajo el Pad
caja1 = sim.getObject('/caja1')
sim.setObjectPosition(caja1, sim.handle_world, [0.4, 0, 0.025] )

In [ ]:
# 2. Bajamos para acercarnos a la caja
# Enviamos al robot la posición deseada
q1_val = 0 * 3.1416/180
q2_val = 0.022
q3_val = 0 * 3.1416/180

sim.setJointTargetPosition(j1, q1_val)
sim.setJointTargetPosition(j2, q2_val)
sim.setJointTargetPosition(j3, q3_val)

In [ ]:
#activamos el actuador
setEffector(1)

In [ ]:
# y subimos
q1_val = 0 * 3.1416/180
q2_val = 0.2
q3_val = 0 * 3.1416/180

sim.setJointTargetPosition(j1, q1_val)
sim.setJointTargetPosition(j2, q2_val)
sim.setJointTargetPosition(j3, q3_val)

In [ ]:
#desactivamos el actuador
setEffector(0)

<h3>3. Obtenemos la cinemática directa a partir de la descripción D-H </h3>


In [ ]:
# Comenzaremos el trabajo desde la descripción de Denavit-Hartenberg para el robot
#  link |     theta     |      d      |      a      |    alpha
# --------------------------------------------------------------
#   1           q1           0.13           0            0
#   2            0             q2         0.2            0
#   3           q3          -0.052        0.2           180
#  -------------------------------------------------------------
#   4           0            0.0496         0           180

In [ ]:
# importamos las librerías requeridas
import sympy

In [ ]:
# Definimos una función para construir las matrices de transformación
# en forma simbóĺica a partir de los parámetros D-H

def MatrixfromDH(theta, d, a, alpha):
    # theta y alpha en radianes
    # d y a en metros
    Rz = sympy.Matrix([[sympy.cos(theta), -sympy.sin(theta), 0, 0],
                   [sympy.sin(theta), sympy.cos(theta), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    tz = sympy.Matrix([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, d],
                   [0, 0, 0, 1]])
    ta = sympy.Matrix([[1, 0, 0, a],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    Rx = sympy.Matrix([[1, 0, 0, 0],
                   [0, sympy.cos(alpha), -sympy.sin(alpha), 0],
                   [0, sympy.sin(alpha), sympy.cos(alpha), 0],
                   [0, 0, 0, 1]])
    return Rz*tz*ta*Rx

In [ ]:
# Obtenemos la matriz de transformación de la base al extremo del robot
q1, q2, q3 = sympy.symbols(['q1', 'q2', 'q3'])
j1_T_j2 = MatrixfromDH(q1, 0.13, 0, 0)
j2_T_j3 = MatrixfromDH(0, q2, 0.2, 0)
j3_T_conn = MatrixfromDH(q3, -0.052, 0.2, sympy.pi)
conn_T_pad = MatrixfromDH(0, 0.0496, 0, sympy.pi)
j1_T_pad = sympy.simplify(j1_T_j2 * j2_T_j3 * j3_T_conn * conn_T_pad)
j1_T_pad

<h3>4. Evaluamos la cinemática inversa </h3>


In [ ]:
#  Primero definimos una función para construir la matriz de rotación
# a partir de una pose, que contiene las coordenadas de traslación
# y los ángulos de Euler que representan la orientación

def MatrixFromPose(x, y, z, alpha, beta, gamma):
    # x, y, z en metros
    # alpha, beta, gamma en radianes
    t = sympy.Matrix([[1, 0, 0, x],
                      [0, 1, 0, y],
                      [0, 0, 1, z],
                      [0, 0, 0, 1]])
    Ra = sympy.Matrix([[1, 0, 0, 0],
                   [0, sympy.cos(alpha), -sympy.sin(alpha), 0],
                   [0, sympy.sin(alpha), sympy.cos(alpha), 0],
                   [0, 0, 0, 1]])
    Rb = sympy.Matrix([[sympy.cos(beta), 0, sympy.sin(beta), 0],
                   [0, 1, 0, 0],
                   [-sympy.sin(beta), 0, sympy.cos(beta), 0],
                   [0, 0, 0, 1]])
    Rg = sympy.Matrix([[sympy.cos(gamma), -sympy.sin(gamma), 0, 0],
                   [sympy.sin(gamma), sympy.cos(gamma), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    T = t*Ra*Rb*Rg
    return T

In [ ]:
# Simplificaremos la matriz de cinemática directa para ocuparnos únicamente de la posición
# Para ello dejaremos la matriz de rotación como la identidad
# Esto es necesario porque tenemos un robot con muy pocos grados de libertad
# Para construir la nueva matriz, utilizaremos la función MatrixFromPose()
x, y, z = j1_T_pad[3], j1_T_pad[7], j1_T_pad[11]
alpha, beta, gamma = 0, 0, 0
T = MatrixFromPose(x, y, z, alpha, beta, gamma)
T

In [ ]:
# Utilizamos esta función para definir la matriz con las coordenadas de destino
# Por ahora no consideraremos las rotaciones
x, y, z = 0.2, 0.2, 0.1
alpha, beta, gamma = 0, 0, 0
D = MatrixFromPose(x, y, z, alpha, beta, gamma)
D

In [ ]:
# Calculamos ahora la cinemática inversa utilizando nsolve.
# La función nsolve de sympy busca valores busca soluciones que
# igualan una expresión a cero. 
# Si requerimos T = D, entonces buscamos los ángulos que hagan T-D = 0

T-D

In [ ]:
# Buscamos una solución relajando la precisión requerida para facilitar el cálculo.
# La precisión por defecto es de 15 cifras significativas.
q = sympy.nsolve((T-D), (q1, q2, q3), (1, 1, 1), prec=6)
q

In [ ]:
# enviamos los ángulos a las articulaciones
sim.setJointTargetPosition(j1, float(q[0]))
sim.setJointTargetPosition(j2, float(q[1]))
sim.setJointTargetPosition(j3, float(q[2]))

In [ ]:
# y verificamos la posición del actuador
sim.getObjectPosition(suctionPad, sim.handle_world)

<h3>5. Ahora lo usamos para tomar una caja </h3>